# Exercise Cleaning
## Eric Rios Soderman

#### This is an exercise in which the American Community Survey (ACS) data will be cleaned, to better facilitate data analysis.

#### Exercise #1

#### Import the ACS data from the url.

In [1]:
# Exercise 1
import pandas as pd
import numpy as np

results = {}

acs = pd.read_stata(
    "https://github.com/nickeubank/MIDS_Data/raw/master/US_AmericanCommunitySurvey/US_ACS_2017_10pct_sample.dta"
)

acs.head()


,year,datanum,serial,cbserial,numprec,subsamp,hhwt,hhtype,cluster,adjust,...,migcounty1,migmet131,vetdisab,diffrem,diffphys,diffmob,diffcare,diffsens,diffeye,diffhear
0,2017,1,177686,2.017001e+12,9,64,55,"female householder, no husband present",2.017002e+12,1.011189,...,0,not in identifiable area,n/a,n/a,n/a,n/a,n/a,no vision or hearing difficulty,no,no
1,2017,1,1200045,2.017001e+12,6,79,25,"male householder, no wife present",2.017012e+12,1.011189,...,0,not in identifiable area,n/a,no cognitive difficulty,no ambulatory difficulty,no independent living difficulty,no,no vision or hearing difficulty,no,no
2,2017,1,70831,2.017000e+12,1 person record,36,57,"male householder, living alone",2.017001e+12,1.011189,...,0,not in identifiable area,n/a,has cognitive difficulty,no ambulatory difficulty,no independent living difficulty,no,no vision or hearing difficulty,no,no
3,2017,1,557128,2.017001e+12,2,10,98,married-couple family household,2.017006e+12,1.011189,...,0,not in identifiable area,n/a,no cognitive difficulty,no ambulatory difficulty,no independent living difficulty,no,no vision or hearing difficulty,no,no
4,2017,1,614890,2.017001e+12,4,96,54,married-couple family household,2.017006e+12,1.011189,...,0,not in identifiable area,n/a,n/a,n/a,n/a,n/a,no vision or hearing difficulty,no,no



Exercise 2

For our exercises today, we’ll focus on age, gender, educ (education), and inctot (total income). Subset your data to those variables, and quickly look at a sample of 10 rows.


In [2]:
# Exercise 2

subset = acs.loc[:, ["age", "sex", "educ", "inctot"]]
subset.sample(10)


,age,sex,educ,inctot
138746,64,male,grade 12,28000
264668,50,female,n/a or no schooling,3000
35137,1,male,n/a or no schooling,9999999
281686,60,female,grade 12,35000
140708,39,male,grade 12,46000
274299,20,female,grade 12,0
291052,13,female,"grade 5, 6, 7, or 8",9999999
228008,14,male,"grade 5, 6, 7, or 8",9999999
53361,29,male,grade 12,59000
13354,3,male,n/a or no schooling,9999999



Exercise 3

As before, all the values of 9999999 have the potential to cause us real problems, so replace all the values of inctot that are 9999999 with np.nan.


In [3]:
subset["inctot"] = subset["inctot"].replace(9999999, np.nan)
subset["inctot"].value_counts(dropna=False)


NaN         53901
0.0         33679
30000.0      4778
50000.0      4414
40000.0      4413
            ...  
70520.0         1
76680.0         1
57760.0         1
200310.0        1
505400.0        1
Name: inctot, Length: 8471, dtype: int64


# Exercise 4

Attempt to calculate the average age of people in our data. What do you get? Why are you getting that error?


In [4]:
# subset.loc[:, 'age'].mean()


In [5]:
subset.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 319004 entries, 0 to 319003
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype   
---  ------  --------------   -----   
 0   age     319004 non-null  category
 1   sex     319004 non-null  category
 2   educ    319004 non-null  category
 3   inctot  265103 non-null  float64 
dtypes: category(3), float64(1)
memory usage: 5.8 MB


#### Age is a category column, not an integer column. This explains why the mean method did not work. Now, the problematic values will be found.

In [6]:
subset.loc[~subset.loc[:, "age"].str.isnumeric(), "age"].value_counts(
    dropna=False
).head(5)


less than 1 year old         3150
90 (90+ in 1980 and 1990)     480
61                              0
70                              0
69                              0
Name: age, dtype: int64

# Exercise 5

We want to be able to calculate things using age, so we need it to be a numeric type. Check the current type of age, and look at all the values of age to figure out why it’s categorical and not numeric. You should find two problematic categories. Store the values of these categories in "ex5_age_young" and "ex5_age_old" (once you find them, it should be clear which is which).


In [7]:
ex5_age_young = "less than 1 year old"
results["ex5_age_young"] = ex5_age_young
ex5_age_old = "90 (90+ in 1980 and 1990)"
results["ex5_age_old"] = ex5_age_old

print(
    f"The problematic values in the age column are {ex5_age_young} and {ex5_age_old}."
)


The problematic values in the age column are less than 1 year old and 90 (90+ in 1980 and 1990).


# Exercise 6

In order to convert age into a numeric variable, we need to replace those problematic entries with values that pandas can later convert into numbers. Pick appropriate substitutions for the existing values and replace the current values.

Hint 1: Categorical variables act like strings, so you might want to use string methods!

Hint 2: Remember that characters like parentheses, pluses, asterisks, etc. are special in Python strings, and you have to escape them if you want them to be interpreted literally!

Hint 3: Because the US Census has been conducted regularly for hundreds of years but exactly how the census has been conducted have occasionally changed, variables are sometimes coded in a way that might be interpreted in different ways for different census years. For example, hypothetically, one might write 90 (90+ in 1980 and 1990) if the Censuses conducted in 1980 and 1990 used to top-code age at 90 (any values over 90 were just coded as 90), but more recent Censuses no longer top-coded age and recorded ages over 90 as the respondents actual age.

In [8]:
subset.loc[:, "age"] = subset.loc[:, "age"].str.replace("less than 1 year old", "0")
subset.loc[:, "age"] = subset.loc[:, "age"].str.replace(
    "90 \(90\+ in 1980 and 1990\)", "90"
)
print("")


C:\Users\ericr\AppData\Local\Temp\ipykernel_11316\455119877.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  subset.loc[:, "age"] = subset.loc[:, "age"].str.replace(


In [9]:
subset["age"].value_counts()


60    4950
54    4821
56    4776
59    4776
58    4734
      ... 
93     476
95     471
92     355
91     227
96      10
Name: age, Length: 97, dtype: int64


# Exercise 7

Now convert age from a categorical to numeric. Calculate the average age amoung this group, and store it in "ex7_avg_age".


In [10]:
subset["age"] = subset["age"].astype("object")
subset["age"] = subset["age"].astype("float")
ex7_avg_age = subset["age"].mean()
results["ex7_avg_age"] = ex7_avg_age

print(f"The average age is {ex7_avg_age:.2f}.")


The average age is 41.30.



# Exercise 8

Let’s now filter out anyone in our data whose age is less than 18. Note that before made age a numeric variable, we couldn’t do this! again, calculate the average age and this time store it in "ex8_avg_age".


In [11]:
# Exercise 8
subset = subset.loc[~(subset.loc[:, "age"] < 18), :]


ex8_avg_age = subset.loc[~(subset.loc[:, "age"] < 18), "age"].mean()
results["ex8_avg_age"] = ex8_avg_age
print(
    f"The average age, after cleaning the records of people younger than 18, is {ex8_avg_age:.2f}"
)


The average age, after cleaning the records of people younger than 18, is 49.76



# Exercise 9

Create an indicator variable for whether each person has at least a college Bachelor’s degree called college_degree. Use this variable to calculate the number of people in the dataset with a college degree. You may assume that to get a college degree you need to complete at least 4 years of college. Save the result as "ex9_num_college".


In [12]:
# To see the possible values
subset.loc[:, "educ"].value_counts()


grade 12                     92576
4 years of college           47212
1 year of college            38746
5+ years of college          29801
2 years of college           20753
grade 5, 6, 7, or 8           5975
grade 11                      5816
grade 10                      4078
n/a or no schooling           3644
grade 9                       3145
nursery school to grade 4     1288
Name: educ, dtype: int64

In [13]:
bach = ["4 years of college", "5+ years of college"]  # At least a bachelor's degree
edu_bool = subset.loc[:, "educ"].isin(bach)
subset["college_degree"] = edu_bool
ex9_num_college = subset.loc[edu_bool, "educ"].count()
results["ex9_num_college"] = ex9_num_college
print(f"The number of people with a college degree is {ex9_num_college:,}.")


The number of people with a college degree is 77,013.



# Exercise 10

Let’s examine how the educational gender gap. Use pd.crosstab to create a cross-tabulation of sex and college_degree. pd.crosstab will give you the number of people who have each combination of sex and college_degree (so in this case, it will give us a 2x2 table with Male and Female as rows, and college_degree True and False as columns, or vice versa.


In [14]:
pd.crosstab(subset.loc[:, "sex"], subset.loc[:, "college_degree"])


college_degree,False,True
sex,,
male,85821,36181
female,90200,40832



# Exercise 11

Counts are kind of hard to interpret. pd.crosstab can also normalize values to give percentages. Look at the pd.crosstab help file to figure out how to normalize the values in the table. Normalize them so that you get the share of men with and without college degree, and the share of women with and without college degrees.

Store the share (between 0 and 1) of men with college degrees in "ex11_share_male_w_degrees", and the share of women with degrees in "ex11_share_female_w_degrees".


In [15]:
x = pd.crosstab(
    subset.loc[:, "sex"], subset.loc[:, "college_degree"], normalize="index"
)
x


college_degree,False,True
sex,,
male,0.703439,0.296561
female,0.688381,0.311619


In [16]:
ex11_share_male_w_degrees = x.iloc[0, 1]
ex11_share_female_w_degrees = x.iloc[1, 1]

results["ex11_share_male_w_degrees"] = ex11_share_male_w_degrees
results["ex11_share_female_w_degrees"] = ex11_share_female_w_degrees

print(
    f"The share of males and females that completed college degrees are {ex11_share_male_w_degrees:.2f} and {ex11_share_female_w_degrees:.2f}"
)


The share of males and females that completed college degrees are 0.30 and 0.31



# Exercise 12

Now, let’s recreate that table for people who are 40 and over and people under 40. Over time, what does this suggest about the absolute difference in the share of men and women earning college degrees? Has it gotten larger, stayed the same, or gotten smaller? Store your answer (either "the absolute difference has increased" or "the absolute difference has decreased") in "ex12_comparing".



In [17]:
forty_older = subset.loc[subset.loc[:, "age"] >= 40]
forty_younger = subset.loc[subset.loc[:, "age"] < 40]
older = pd.crosstab(
    forty_older.loc[:, "sex"], forty_older.loc[:, "college_degree"], normalize="index"
)
younger = pd.crosstab(
    forty_younger.loc[:, "sex"],
    forty_younger.loc[:, "college_degree"],
    normalize="index",
)
older


college_degree,False,True
sex,,
male,0.682123,0.317877
female,0.699144,0.300856


In [18]:
younger


college_degree,False,True
sex,,
male,0.743143,0.256857
female,0.665710,0.334290


In [19]:
ex12_comparing = "the absolute difference has increased"
results["ex12_comparing"] = ex12_comparing


# Exercise 13
In words, what is causing the change noted in Exercise 12 (i.e., looking at the tables above, tell me a story about Men and Women’s College attainment).

With the passage of time, it is evident that less men are pursuing bachelor's degrees and graduate degrees. For the group that is 40 and older (those born on 1976 and earlier), the men had a slightly higher share of college attainment rate than women. For those 40 and below, the gap grows to from hardly a percentage point to approximately 8 percentage points, where men are earning less college degrees. In addition, due to having prior knowledge of this declining trend, I decided to filter and crosstab for the group between 40 and 22. My reasoning is due to skewing by young undergraduate students who are yet to graduate. Predictably, 31% of men in this range were earning degrees versus the 41% of women. 

Recent trends have been very telling of this phenomenon. Most deserters in high school are men, for example. Therefore, there is an underlying root issue that is causing young men to value education less. In addition, there are industries that do not require men to attain degrees in order to work, such as construction, as well as specialized, short-term projects for certain jobs (i.e. electricians and car repairs). In contrast, focus on female education has incremented due to a cultural shift for wanting more female, college-educated professionals. However, interestingly enough, many of these "strength" or hard labor industries are still highly populated by men, and recent studies have shown that females tend to follow the college route and avoid these types of hard labor industries altogether. To conclude, these are some factors influencing the current trends, but there are many more due to the multiple influences affecting them.  

In [20]:
thirty_22 = subset.loc[(subset.loc[:, "age"] < 40) & (subset.loc[:, "age"] >= 22)]
pd.crosstab(
    thirty_22.loc[:, "sex"], thirty_22.loc[:, "college_degree"], normalize="index"
)


college_degree,False,True
sex,,
male,0.682440,0.317560
female,0.588905,0.411095


In [21]:
assert set(results.keys()) == {
    "ex5_age_young",
    "ex5_age_old",
    "ex7_avg_age",
    "ex8_avg_age",
    "ex9_num_college",
    "ex11_share_male_w_degrees",
    "ex11_share_female_w_degrees",
    "ex12_comparing",
}
